**File reading**
***

In [1]:
import pandas as pd
# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = pd.read_csv('docs/simpsons_dataset.csv').dropna()

# imprima las primeras 5 filas
df.head()

#Cambiamos las etiquetas texutales por etiquetas numericas, esto es una buena practica cuando se construyen modelo supervisados
# df['label'] = df.label.map({'ham':0, 'spam':1})
print(df.shape)
df.head(20) # returns (rows, columns)

(131853, 2)


,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...
5,Martin Prince,I don't think there's anything left to say.
6,Edna Krabappel-Flanders,Bart?
7,Bart Simpson,Victory party under the slide!
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
10,Landlady,"Hey, hey, he Moved out this morning. He must h..."


## Text Pre Processing for embeddings

In [2]:
import gzip
import gensim
import logging

KeyboardInterrupt: 

### BOW

In [ ]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess(line)

In [ ]:
from gensim.utils import simple_preprocess

def preproccess(line):
    return simple_preprocess(line)



df['proccesed'] = df['spoken_words'].apply(preproccess)
df.head(10)

In [ ]:
documents = df['proccesed'].to_numpy()

In [ ]:
model_skipgram = gensim.models.Word2Vec(documents, vector_size=150, window=10, min_count=2, workers=10, sg=1)
model_skipgram.train(documents,total_examples=len(documents),epochs=10)

In [ ]:
model_skipgram.save("word2vec150.model")

In [ ]:
print(model_skipgram.wv['morning'])

In [ ]:
model_100 = gensim.models.Word2Vec (documents, vector_size=100, window=10, min_count=2, workers=10, sg=1)
model_100.train(documents,total_examples=len(documents),epochs=10)

In [ ]:
model_100.save("word2vec100.model")

In [ ]:
print(model_100.wv['morning'])

In [ ]:
model_50 = gensim.models.Word2Vec(documents, vector_size=50, window=10, min_count=2, workers=10, sg=1)
model_50.train(documents,total_examples=len(documents),epochs=10)

In [ ]:
model_50.save("word2vec50.model")


In [ ]:
df.shape

## Read


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:

model50 = gensim.models.Word2Vec.load("word2vec50.model")
model100 = gensim.models.Word2Vec.load("word2vec100.model")
model150 = gensim.models.Word2Vec.load("word2vec150.model")

In [ ]:
model50.wv.similar_by_word('homer')

In [ ]:
def display_closestwords_tsnescatterplot(model, word, n):
    
    arr = np.empty((0,n), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.wv.similar_by_word(word)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model.wv[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model.wv[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.savefig(f'./{word}_{n}.jpg')
    plt.show()

## Graphs and analysis
***

**Vector dimension:** 50

In [ ]:
display_closestwords_tsnescatterplot(model50, 'bart', 50)

In [ ]:
display_closestwords_tsnescatterplot(model50, 'homer', 50)

**Vector dimension:** 100

In [ ]:
display_closestwords_tsnescatterplot(model100, 'bart', 100)

In [ ]:
display_closestwords_tsnescatterplot(model100, 'lisa', 100)

**Vector size:** 150

In [ ]:
display_closestwords_tsnescatterplot(model150, 'maggie', 150)

In [ ]:
display_closestwords_tsnescatterplot(model150, 'lisa', 150)

# Redes neuronales

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from sklearn.model_selection import train_test_split

In [2]:
from sklearn.preprocessing import OneHotEncoder

In [3]:

# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df_0 = pd.read_csv('docs/simpsons_dataset.csv').dropna()

target_simpsons = [
    'Lisa Simpson', 
    'Bart Simpson',
    'Homer Simpson',
    'Marge Simpson',
    'Maggie Simpson',
    'Grampa Simpson'
]
df = df_0.loc[df_0['raw_character_text'].isin(target_simpsons)]

In [4]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df[['raw_character_text']])
print(enc.categories_)

[array(['Bart Simpson', 'Grampa Simpson', 'Homer Simpson', 'Lisa Simpson',
       'Maggie Simpson', 'Marge Simpson'], dtype=object)]


In [5]:
vectorize_layer = TextVectorization(
    ngrams=None, 
    max_tokens=None, vocabulary=None,
    output_mode='binary', output_sequence_length=None, pad_to_max_tokens=True, 
)

In [6]:
# df_yelp = df[df['source'] == 'yelp']
sentences = df['spoken_words'].values
# sentences =

y = enc.transform(df[['raw_character_text']]).toarray()
sentences_train_val, sentences_test, y_train_val, y_test = train_test_split(sentences, y, test_size=0.20, random_state=1000)
sentences_train, sentences_val, y_train, y_val = train_test_split(sentences_train_val, y_train_val, test_size=0.25, random_state=1000)

# Once we have our handles, we format the datasets in a Keras-fit compatible
# format: a tuple of the form (text_data, label).
train_dataset = tf.data.Dataset.from_tensor_slices((sentences_train,y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((sentences_test,y_test))

# We also create a dataset with only the textual data in it. This will be used
# to build our vocabulary later on.
text_dataset = tf.data.Dataset.from_tensor_slices((sentences_train))

In [7]:
# list(enc.get_feature_names())
len(vectorize_layer.get_vocabulary())

0

In [8]:
# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary
vectorize_layer.adapt(text_dataset.batch(64))

In [9]:
##LAYERS

# Start by creating an explicit input layer. It needs to have a shape of  
# (1,) (because we need to guarantee that there is exactly one doc
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text') 

# The first layer in our model is the vectorization layer. After this layer,
# we have a tensor of shape (batch_size, max_len) containing vocab indices.

x = vectorize_layer(text_input) 
primera_capa = layers.Dense(10, activation='relu')(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense( len(target_simpsons), activation='softmax', name='predictions')(primera_capa) 
model = tf.keras.Model(text_input, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
# https://keras.io/api/losses/
# https://keras.io/api/optimizers/
# Ojo con las metricas ya que el entrenamiento es en batches
# https://keras.io/api/models/model_training_apis/
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [14]:
batch_size = 32
epochs = 20

# Fit the model using the train and test datasets.
model.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/20
1249/1249 [==============================] - 13s 9ms/step - loss: 0.4612 - accuracy: 0.4197 - val_loss: 0.3456 - val_accuracy: 0.4775

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text (InputLayer)            [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 20742)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                207430    
_________________________________________________________________
predictions (Dense)          (None, 6)                 66        
Total params: 207,496
Trainable params: 207,496
Non-trainable params: 0
_________________________________________________________________


### Redes neuronales

In [10]:
from keras.models import Sequential

In [11]:
vocab_size = len(vectorize_layer.get_vocabulary())
vocab_size

20742

In [15]:
embedding_dim = 50
# int
# the: 60
# goog 824
# ....
# [60, 824]
# Feed Forward
modelemb = Sequential([
  vectorize_layer,
  layers.Embedding(vocab_size, embedding_dim, name="embedding"),
  layers.GlobalAveragePooling1D(), #Esta capa es para reducir Features. Soluciona el problema de distintos tamaños de dialogos.
  layers.Dense(10, activation='relu'),
  layers.Dense(len(target_simpsons), activation='softmax')
])

In [16]:
modelemb.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])

In [17]:
batch_size = 32
epochs = 10

modelemb.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
1249/1249 [==============================] - 117s 93ms/step - loss: 1.4604 - accuracy: 0.4186 - val_loss: 1.4040 - val_accuracy: 0.4155
Epoch 2/10
1249/1249 [==============================] - 117s 94ms/step - loss: 1.3982 - accuracy: 0.4186 - val_loss: 1.4038 - val_accuracy: 0.4155
Epoch 3/10
1249/1249 [==============================] - 120s 96ms/step - loss: 1.3980 - accuracy: 0.4186 - val_loss: 1.4037 - val_accuracy: 0.4155
Epoch 4/10
1249/1249 [==============================] - 115s 92ms/step - loss: 1.3979 - accuracy: 0.4186 - val_loss: 1.4037 - val_accuracy: 0.4155
Epoch 5/10
1249/1249 [==============================] - 114s 92ms/step - loss: 1.3979 - accuracy: 0.4186 - val_loss: 1.4037 - val_accuracy: 0.4155
Epoch 6/10
1249/1249 [==============================] - 114s 91ms/step - loss: 1.3978 - accuracy: 0.4186 - val_loss: 1.4036 - val_accuracy: 0.4155
Epoch 7/10
1249/1249 [==============================] - 113s 91ms/step - loss: 1.3978 - accuracy: 0.4186 - val_loss: 1

In [18]:
modelemb.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 20742)             0         
_________________________________________________________________
embedding (Embedding)        (None, 20742, 50)         1037100   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 66        
Total params: 1,037,676
Trainable params: 1,037,676
Non-trainable params: 0
_________________________________________________________________
